https://github.com/SainathAthineni/dl

https://watch.wave.video/4TYzDT18TWGu6A4w

### Table: Contributions of Team Members

| **Contribution**                | **Prabin Awal**    | **Harkirat Singh** |
|---------------------------------|--------------------|--------------------|
| **Image Display**               |                    | X                  |
| **Data Splitting**              |                    | X                  |
| **Model Building**              | X                  |                    |
| **Model Tuning**                | X                  |                    |
| **Model Saving**                | X                  |                    |


In [30]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam
from flask import Flask, request, jsonify
import streamlit as st
import requests
from PIL import Image
import keras_tuner as kt
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
from sklearn.model_selection import train_test_split
import shutil

In [2]:
data_dir = "/Users/prabinawal/Downloads/FaceExpressions/Dataset"
categories = os.listdir(data_dir)

In [3]:
plt.figure(figsize=(15, 3))

for i, category in enumerate(categories, start=1):
    category_path = os.path.join(data_dir, category)
    if not os.path.isdir(category_path):
        continue
    
    image_files = [file for file in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, file)) and not file.startswith('.DS_Store')]
    if image_files:
        random_image = random.choice(image_files)
        image_path = os.path.join(category_path, random_image)
        
        try:
            img = mpimg.imread(image_path)
            plt.subplot(1, len(categories), i)
            plt.imshow(img)
            plt.title(category)
            plt.axis('off')
        except Exception as e:
            print(f"Could not load image {image_path}. Error: {e}")

plt.tight_layout()
plt.show()

/var/folders/z1/yvqnclv90j92_k2wmvyt59400000gn/T/ipykernel_34510/4161241626.py:23: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [4]:
# Splitting data into train, val, and test
categories = [category for category in categories if os.path.isdir(os.path.join(data_dir, category))]
train_files, val_files, test_files = [], [], []

In [5]:
for category in categories:
    category_path = os.path.join(data_dir, category)
    if not os.path.isdir(category_path):
        continue
    image_files = [file for file in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, file)) and not file.startswith('.DS_Store')]
    if image_files:
        train, test_val = train_test_split(image_files, test_size=0.4, random_state=42)
        val, test = train_test_split(test_val, test_size=0.5, random_state=42)
        train_files.extend([(file, category) for file in train])
        val_files.extend([(file, category) for file in val])
        test_files.extend([(file, category) for file in test])

In [6]:
print(f"Number of training files: {len(train_files)}")
print(f"Number of validation files: {len(val_files)}")
print(f"Number of test files: {len(test_files)}")

Number of training files: 0
Number of validation files: 0
Number of test files: 0


In [7]:
# Create directories for train, val, and test
split_dirs = {'train': train_files, 'val': val_files, 'test': test_files}

for split, files in split_dirs.items():
    split_dir = os.path.join(data_dir, split)
    if not os.path.exists(split_dir):
        os.makedirs(split_dir)
    for category in categories:
        category_split_dir = os.path.join(split_dir, category)
        if not os.path.exists(category_split_dir):
            os.makedirs(category_split_dir)
    for file, category in files:
        src_path = os.path.join(data_dir, category, file)
        dst_path = os.path.join(data_dir, split, category, file)
        shutil.move(src_path, dst_path)

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data directories
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

In [9]:
batch_size = 32
image_size = (224, 224)

In [10]:
# Data generators
train_datagen = ImageDataGenerator(rescale=0.2)
val_test_datagen = ImageDataGenerator(rescale=0.2)

In [11]:
# Flow data from directories
train_data = train_datagen.flow_from_directory(train_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical')
val_data = val_test_datagen.flow_from_directory(val_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical')
test_data = val_test_datagen.flow_from_directory(test_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical')

Found 2608 images belonging to 9 classes.
Found 869 images belonging to 9 classes.
Found 872 images belonging to 9 classes.


In [12]:
num_classes = train_data.num_classes
print("Number of classes:", num_classes)

Number of classes: 9


In [13]:
def build_model(hp):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = Flatten()(x)
    x = Dense(units=hp.Int('units', min_value=512, max_value=1024, step=128), activation='relu')(x)
    x = Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1))(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(
        optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [14]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',  
    max_trials=10,
    directory='my_dir',
    project_name='face_expression_tuning'
)


Reloading Tuner from my_dir/face_expression_tuning/tuner0.json


In [15]:
tuner.search(train_data, epochs=2, validation_data=val_data)
best_model = tuner.get_best_models(num_models=1)[0]

Trial 10 Complete [00h 29m 34s]
val_accuracy: 0.6329113841056824

Best val_accuracy So Far: 0.6985040307044983
Total elapsed time: 05h 07m 33s


/Users/prabinawal/anaconda3/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [26]:
best_model.save('/Users/prabinawal/streamlit_app/best_model.h5')

In [27]:
model = tf.keras.models.load_model('/Users/prabinawal/flask_app/best_model.h5')